In [5]:
%matplotlib inline
from utils import read, createGraph, genders, functionals
import networkx as nx
from glob import iglob as glob
from os.path import exists, basename
import pandas as pd
from collections import defaultdict

In [3]:
th = 2
data_dir = "./data/by_movie/*"
info_dir = "./data/speakersWithCharacterInfo/"

In [4]:
def graphAnalytics(G):
        
    #Gender
    males = G.subgraph([i for i in G.nodes() if G.node[i]['gender'] == 'male'])
    females = G.subgraph([i for i in G.nodes() if G.node[i]['gender'] == 'female'])
    
    #Basic measures
    nodes, edges = len(G.nodes()), len(G.edges())
    nodes_m, edges_m = len(males.nodes()), len(males.edges())
    nodes_f, edges_f = len(females.nodes()), len(females.edges())
    res = [nodes, edges, nodes_m, edges_m, nodes_f, edges_f]
    
    
    #Degree Measures
    f_deg, f_deg_m, f_deg_f = functionals([v for _,v in nx.degree(G).items()]),\
                              functionals([v for _,v in nx.degree(males).items()]),\
                              functionals([v for _,v in nx.degree(females).items()])
    res.extend(f_deg+f_deg_m+f_deg_f)
            
    #Distance Measures
    paths = [[v for _,v in vals.items()] for _,vals in nx.shortest_path_length(G).items()]
    paths_m = [[v for _,v in vals.items()] for _,vals in nx.shortest_path_length(males).items()]
    paths_f = [[v for _,v in vals.items()] for _,vals in nx.shortest_path_length(females).items()]
    
    
    
    return res

# For all scripts

In [ ]:
data = []
for script in glob(data_dir):
    try:
        char_list, adj = read(script)
        
        gender_file = info_dir + basename(script).replace(".", "_") + ".txt"
        if exists(gender_file):
            gens = genders(gender_file)
        else:
            gens = defaultdict(lambda: 'unknown')
        
        G = createGraph(char_list, adj, gens)
        func_G = graphAnalytics(G)
        
        
        
        data.append([script]+func_G)
    except:
        pass

In [ ]:
data = pd.DataFrame(data, columns= ["script", "nodes", "edges", "nodes_males", "edges_males", "nodes_females", "edges_females",
                                    "degree_min", "degree_mean", "degree_median", "degree_max",
                                    "male_degree_min", "male_degree_mean", "male_degree_median", "male_degree_max",
                                    "female_degree_min", "degree_mean", "degree_median", "degree_max",])
data.head()

In [ ]:
len(data)

### Save dataframe

In [ ]:
data.to_pickle('./data/dataframe.pkl')